In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 15 17:22:08 2017

@author: cnbyb
"""

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
tf.Session(config=config)
import numpy as np

def g(x):
    return tf.nn.relu(x)**3
    #return 1.0507*(tf.nn.relu(x)+1.6733*tf.exp(-tf.nn.relu(-x))-1.6733)
    
def block(x,size,scope):
    initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1)
    with tf.variable_scope(scope):
        W1=tf.get_variable('w1',shape=size,dtype='float',initializer=initializer)
        b1=tf.get_variable('b1',dtype='float',initializer=tf.zeros([size[1]]))
        W2=tf.get_variable('w2',shape=[size[1],size[1]],dtype='float',initializer=initializer)
        b2=tf.get_variable('b2',dtype='float',initializer=tf.zeros([size[1]]))
        y=tf.matmul(x,W1)+b1
        y=g(y)
        y=tf.matmul(y,W2)+b2
        
        if (size[0]<size[1]):
            y=g(y)+tf.concat([x,tf.matmul(x,tf.zeros([2,size[1]-size[0]]))],axis=-1)
        elif (size[0]==size[1]):
            y=g(y)+x
        else:
            w_res=tf.get_variable('w_res',shape=[size[0],size[1]],dtype=tf.float32)
            y=g(y)+tf.matmul(x,w_res)
        
    return y

def func(x,is_training):
    size=10
    y=block(x,[2,size],'block1')
    y=block(y,[size,size],'block2')
    y=block(y,[size,size],'block3')
    y=block(y,[size,size],'block4')
    y=block(y,[size,size],'block5')
    #y=block(y,[size,size],'block6')
    #y=block(y,[size,size],'block7')
    #y=block(y,[size,size],'block8')
    W=tf.get_variable("w", shape=[size,1], dtype='float')
    b=tf.get_variable("b", shape=[1], dtype='float')
    u=tf.matmul(y,W)+b
    return u
def pfunc(x):
    return x[:,0]*x[:,1]

#sample node points in Omega, nodeb*4 points on the border of Omega
def get_batch(node,nodeb):
    x=np.random.random([node,2])
    xblist=[]
    for d in range(2):
        xb0=np.random.random([nodeb,2])
        xb0[:,d]=0.0
        xb1=np.random.random([nodeb,2])
        xb1[:,d]=1.0
        xblist.append(xb0)
        xblist.append(xb1)
    return x,np.concatenate(xblist,axis=0)

node=500
nodeb=100
#x=tf.placeholder(tf.float32,[(2*node+1)**2,2])
x=tf.placeholder(tf.float32,[None,2])
#xb=tf.placeholder(tf.float32,[nodeb*5,2])
xb=tf.placeholder(tf.float32,[None,2])
is_training=tf.placeholder(tf.bool)
with tf.variable_scope('y') as scope:
    u=func(x,is_training)
    scope.reuse_variables()
    ub=func(xb,is_training)
pub=pfunc(xb)
pu=pfunc(x)
ux=tf.gradients(u,x)[0]
#beta=tf.get_variable(name='beta',1e2,trainable=False)
beta=5e2
Iusum=tf.reduce_mean(tf.reduce_sum(ux*ux,axis=1))/2
loss=Iusum+beta*tf.reduce_mean((ub[:,0]-pub)**2)
global_step=tf.get_variable(name='global_step',initializer=0,trainable=False)
lr=tf.train.exponential_decay(1e-2,global_step,10000,0.5,staircase=True)
op=tf.assign_add(global_step,1)
#opt=tf.train.AdamOptimizer(lr,0.99)
opt=tf.train.AdamOptimizer(lr)
with tf.control_dependencies([op]):
    grad=opt.compute_gradients(loss)
    train_op=opt.apply_gradients(grad)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(50000):
    x_batch,xb_batch=get_batch(node,nodeb)
    sess.run(train_op,feed_dict={x:x_batch,xb:xb_batch,is_training:True})
    if ((i%100)==0):
        #每一百次输出：迭代次数；I(u)减去正则项的值；I(u)的值
        print(i,sess.run([Iusum,loss],feed_dict={x:x_batch,xb:xb_batch,is_training:False}))
node=10240
xgrid=np.random.random([node,2])

u_solve=sess.run(u,feed_dict={x:xgrid,is_training:False})
pu=pfunc(xgrid)
#pu=np.reshape(pfunc_new(xgrid),[2*node+1,2*node+1])

#相对误差
print(np.linalg.norm(np.reshape(u_solve[:,0]-pu,[-1]),ord=2)/np.linalg.norm(np.reshape(pu,[-1]),ord=2))




D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 [0.0489372, 2251.9116]
100 [1.3408374, 109.5312]
200 [0.71674806, 31.149149]
300 [0.51314634, 17.422426]
400 [0.4307174, 12.459279]
500 [0.3835897, 10.131487]
600 [0.35750282, 8.453099]
700 [0.3500016, 7.1058407]
800 [0.3387824, 6.2885966]
900 [0.3579207, 5.3872237]
1000 [0.3519293, 4.153334]
1100 [0.32428595, 3.7236655]
1200 [0.34535444, 3.4040275]
1300 [0.33647296, 2.793117]
1400 [0.37668058, 2.2461853]
1500 [0.35690063, 2.1451614]
1600 [0.38592485, 1.6712991]
1700 [0.35238072, 1.7256234]
1800 [0.35094303, 1.4769022]
1900 [0.34861717, 2.068184]
2000 [0.3915445, 1.2703003]
2100 [0.36014903, 1.180649]
2200 [0.39357617, 1.0562294]
2300 [0.37770152, 1.2787697]
2400 [0.35387555, 0.96098924]
2500 [0.37006262, 0.95827997]
2600 [0.35106736, 1.1390096]
2700 [0.33867633, 0.97102076]
2800 [0.36643136, 0.86600685]
2900 [0.34957248, 1.1007197]
3000 [0.438564, 1.2023017]
3100 [0.35867587, 0.70752627]
3200 [0.37278455, 0.6620363]
3300 [0.3672515, 0.7118603]
3400 [0.3237996, 0.5638939]
3500 [0.358

27600 [0.32859853, 0.3323753]
27700 [0.33650607, 0.34032065]
27800 [0.3412314, 0.34407678]
27900 [0.32561415, 0.33359623]
28000 [0.33656973, 0.33915597]
28100 [0.32442823, 0.32624462]
28200 [0.32865822, 0.3302213]
28300 [0.3351457, 0.33932173]
28400 [0.3047639, 0.3067891]
28500 [0.33216622, 0.33617735]
28600 [0.33820045, 0.3465837]
28700 [0.31922036, 0.32336098]
28800 [0.31633753, 0.3190448]
28900 [0.3295088, 0.33220193]
29000 [0.3367252, 0.34017843]
29100 [0.31932873, 0.32210442]
29200 [0.32682106, 0.33223322]
29300 [0.3338894, 0.33639362]
29400 [0.3210117, 0.32435822]
29500 [0.34385473, 0.3467472]
29600 [0.31657195, 0.3183865]
29700 [0.3331219, 0.3369099]
29800 [0.326617, 0.33032608]
29900 [0.32873335, 0.3330458]
30000 [0.32527024, 0.32817918]
30100 [0.34570238, 0.3490244]
30200 [0.32844016, 0.33070153]
30300 [0.3245404, 0.3287512]
30400 [0.31203496, 0.31442484]
30500 [0.3286987, 0.3310739]
30600 [0.35407588, 0.35696954]
30700 [0.32231504, 0.32432196]
30800 [0.32798138, 0.33127365]
3